# CatBoost Optimized Model

Train CatBoost with optimized hyperparameters for best performance.

## 1. Setup

In [1]:
# Core data science libraries
import pandas as pd
import warnings

# CatBoost specific imports
from catboost import (
    Pool,
)

# Project utilities
from test_case_fairmoney.utils.constants import (
    PATH_DATA_PROCESSED,
    PATH_MODELS,
    PATH_DATA_COMPRESSED,
)
from test_case_fairmoney.utils.model_validation import plot_AUC, plot_PR
from test_case_fairmoney.utils.model_training import (
    train_catboost_model,
    analyze_feature_importance_cb,
)

# Configuration
pd.set_option("display.max_rows", 300)
pd.set_option("display.max_columns", 800)
pd.set_option("display.float_format", "{:.4f}".format)
warnings.filterwarnings("ignore")

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


## 2. Load Data

In [2]:
# Load preprocessed datasets
print("📊 Loading preprocessed datasets...")
df_train = pd.read_parquet(PATH_DATA_COMPRESSED / "df_train.parquet")
df_val = pd.read_parquet(PATH_DATA_COMPRESSED / "df_val.parquet")
df_test = pd.read_parquet(PATH_DATA_COMPRESSED / "df_test.parquet")

print(f"✅ Datasets loaded successfully!")
print(f"   📈 Train set: {df_train.shape[0]:,} samples, {df_train.shape[1]} features")
print(f"   📊 Validation set: {df_val.shape[0]:,} samples, {df_val.shape[1]} features")
print(f"   🎯 Test set: {df_test.shape[0]:,} samples, {df_test.shape[1]} features")

# Verify class distribution consistency
print(f"\n🎯 Class Distribution Verification:")
for name, df in [("Train", df_train), ("Validation", df_val), ("Test", df_test)]:
    default_rate = df["default"].mean()
    print(f"   {name:>10}: {default_rate:.4f} ({default_rate*100:.2f}%)")

📊 Loading preprocessed datasets...
✅ Datasets loaded successfully!
   📈 Train set: 600 samples, 27 features
   📊 Validation set: 200 samples, 27 features
   🎯 Test set: 200 samples, 27 features

🎯 Class Distribution Verification:
        Train: 0.3000 (30.00%)
   Validation: 0.3000 (30.00%)
         Test: 0.3000 (30.00%)


## 3. Define Features

In [3]:
# Define feature sets for CatBoost
feat_num = ["checking_balance", "amount", "savings_balance", "employment_months"]
feat_cat = [
    "credit_history",
    "personal_status",
    "property",
    "installment_plan",
    "is_checking_negative",
]
target = "default"

print(f"🔢 Numerical features ({len(feat_num)}): {feat_num}")
print(f"📝 Categorical features ({len(feat_cat)}): {feat_cat}")
print(f"🎯 Target variable: {target}")

# Handle missing values in categorical features
print(f"\n🔧 Preprocessing categorical features...")
for feature in feat_cat:
    # Fill missing values with 'Missing' category
    train_missing = df_train[feature].isnull().sum()
    val_missing = df_val[feature].isnull().sum()
    test_missing = df_test[feature].isnull().sum()

    if train_missing + val_missing + test_missing > 0:
        print(
            f"   {feature}: {train_missing + val_missing + test_missing} missing values filled"
        )

    df_train[feature] = df_train[feature].fillna("Missing")
    df_val[feature] = df_val[feature].fillna("Missing")
    df_test[feature] = df_test[feature].fillna("Missing")

print("✅ Feature preprocessing completed")

🔢 Numerical features (4): ['checking_balance', 'amount', 'savings_balance', 'employment_months']
📝 Categorical features (5): ['credit_history', 'personal_status', 'property', 'installment_plan', 'is_checking_negative']
🎯 Target variable: default

🔧 Preprocessing categorical features...
   personal_status: 310 missing values filled
✅ Feature preprocessing completed


## 4. Create CatBoost Pools

In [4]:
# Create CatBoost Pool objects with categorical feature specification
print("🏊 Creating CatBoost Pools...")

train_pool = Pool(
    data=df_train[feat_cat + feat_num],
    label=df_train[target],
    cat_features=feat_cat,
)

val_pool = Pool(
    data=df_val[feat_cat + feat_num],
    label=df_val[target],
    cat_features=feat_cat,
)

test_pool = Pool(
    data=df_test[feat_cat + feat_num],
    label=df_test[target],
    cat_features=feat_cat,
)

print(f"✅ Pools created successfully!")
print(f"   🏊 Train pool: {train_pool.num_row()} samples")
print(f"   🏊 Validation pool: {val_pool.num_row()} samples")
print(f"   🏊 Test pool: {test_pool.num_row()} samples")
print(f"   📊 Features per pool: {train_pool.num_col()}")
print(f"   📝 Categorical features: {len(feat_cat)}")

🏊 Creating CatBoost Pools...
✅ Pools created successfully!
   🏊 Train pool: 600 samples
   🏊 Validation pool: 200 samples
   🏊 Test pool: 200 samples
   📊 Features per pool: 9
   📝 Categorical features: 5


## 5. Train Optimized Model

In [5]:
# Define base and optimized parameters
print("⚙️ Configuring model parameters...")

# Base parameters for stable training
BASE_PARAMS = {
    "learning_rate": 0.1,
    "loss_function": "Logloss",
    "iterations": 200,
    "depth": 5,
    "eval_metric": "AUC:use_weights=False",
    "l2_leaf_reg": 5,
    "auto_class_weights": "Balanced",  # Handle class imbalance
    "random_seed": 42,
    "boosting_type": "Ordered",
    "bootstrap_type": "MVS",
    "verbose": False,
}

# Optimized parameters from hyperparameter tuning
OPTIMIZED_PARAMS = {
    "learning_rate": 0.02814097068151777,
    "depth": 6,
    "l2_leaf_reg": 27.45959338993318,
    "min_data_in_leaf": 5,
    "random_strength": 4.360675824806039,
    "boosting_type": "Ordered",
    "bootstrap_type": "Bayesian",
    "bagging_temperature": 6.457046635415198,
}

# Training configuration
custom_training_params = {
    "use_best_model": True,
    "early_stopping_rounds": 50,
    "plot": True,
}

# Merge parameters
final_params = {**BASE_PARAMS, **OPTIMIZED_PARAMS}

print("📋 Final Model Configuration:")
for key, value in final_params.items():
    print(f"   {key}: {value}")

print(f"\n🚀 Starting model training...")

⚙️ Configuring model parameters...
📋 Final Model Configuration:
   learning_rate: 0.02814097068151777
   loss_function: Logloss
   iterations: 200
   depth: 6
   eval_metric: AUC:use_weights=False
   l2_leaf_reg: 27.45959338993318
   auto_class_weights: Balanced
   random_seed: 42
   boosting_type: Ordered
   bootstrap_type: Bayesian
   verbose: False
   min_data_in_leaf: 5
   random_strength: 4.360675824806039
   bagging_temperature: 6.457046635415198

🚀 Starting model training...


In [6]:
# Train the optimized CatBoost model
results_custom = train_catboost_model(
    train_pool=train_pool,
    val_pool=val_pool,
    test_pool=test_pool,
    model_params=final_params,
    training_params=custom_training_params,
    verbose=True,
)

print("✅ Model training completed!")


🚀 CATBOOST MODEL TRAINING

📋 Model Configuration:
   Iterations:       200
   Learning Rate:    0.02814097068151777
   Tree Depth:       6
   Loss Function:    Logloss
   Class Weights:    Balanced

📊 Dataset Information:
   Training samples:    600
   Validation samples:  200
   Test samples:        200
   Total features:      9

⚖️  Class Distribution (Train):
   Positive class: 30.00%
   Negative class: 70.00%

🎯 Initializing CatBoost model...

🏋️  Training model...
   Early stopping:   50
   Best model:       Yes
------------------------------------------------------------


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


🔮 Generating predictions...

📊 PERFORMANCE METRICS

🎯 Train Set:
   ROC-AUC:      0.8289
   PR-AUC:       0.7168
   Baseline:     0.3000
   Lift:         +138.9% over random

🎯 Validation Set:
   ROC-AUC:      0.7251
   PR-AUC:       0.5426
   Baseline:     0.3000
   Lift:         +80.9% over random

🎯 Test Set:
   ROC-AUC:      0.7910
   PR-AUC:       0.6208
   Baseline:     0.3000
   Lift:         +106.9% over random

✅ MODEL TRAINING COMPLETED

🏆 Best Performance (Validation):
   ROC-AUC: 0.7251
   PR-AUC:  0.5426

🔝 Top 10 Most Important Features:
   checking_balance                  43.40  (numerical)
   credit_history                    12.83  (categorical)
   is_checking_negative              12.19  (categorical)
   amount                            10.73  (numerical)
   savings_balance                   10.65  (numerical)
   employment_months                  4.81  (numerical)
   property                           4.73  (categorical)
   personal_status                    0.66 

## 6. Performance Analysis

In [7]:
# Extract results from training
model = results_custom["model"]
df_metrics = results_custom["metrics"]
df_feature_importance = results_custom["feature_importance"]
predictions = results_custom["predictions"]

print("📊 Model Training Results Summary:")
print("=" * 50)

# Display metrics for each dataset
for dataset in ["train", "val", "test"]:
    if dataset in df_metrics.columns:
        metrics = df_metrics[dataset].iloc[-1]  # Get final metrics
        print(f"\n🎯 {dataset.capitalize()} Set Performance:")
        for metric_name, value in metrics.items():
            print(f"   {metric_name}: {value:.4f}")

# Training iterations info
print(f"\n🔄 Training Details:")
print(f"   Total iterations: {len(df_metrics)}")
print(f"   Best iteration: {model.get_best_iteration()}")
print(f"   Model depth: {model.get_param('depth')}")
print(f"   Learning rate: {model.get_param('learning_rate')}")

📊 Model Training Results Summary:

🔄 Training Details:
   Total iterations: 3
   Best iteration: 14
   Model depth: 6
   Learning rate: 0.02814097068151777


## 7. Feature Importance

In [8]:
# Analyze feature importance
print("🔍 Feature Importance Analysis:")
print("=" * 40)

analyze_feature_importance_cb(model=model, pool=train_pool, top_n=10)

🔍 Feature Importance Analysis:

📊 TOP 10 MOST IMPORTANT FEATURES
   checking_balance                  45.36
   credit_history                    11.68
   savings_balance                   11.10
   is_checking_negative              10.96
   amount                            10.03
   employment_months                  4.69
   property                           4.45
   personal_status                    1.22
   installment_plan                   0.52



feature  importance
5      checking_balance     45.3602
0        credit_history     11.6787
7       savings_balance     11.0956
4  is_checking_negative     10.9567
6                amount     10.0327
8     employment_months      4.6880
2              property      4.4521
1       personal_status      1.2192
3      installment_plan      0.5167

## 8. Save Model

In [9]:
# Optional: Save the optimized model
# Uncomment to save the trained model for future use

print("💾 Saving Optimized CatBoost Model...")
model_save_path = PATH_MODELS / "catboost_optimized.cbm"
model.save_model(model_save_path)
print(f"✅ Model saved to: {model_save_path}")

💾 Saving Optimized CatBoost Model...
✅ Model saved to: /Users/phamkythoai/Desktop/Data_Science/test_case_fairmoney/project_data/models/catboost_optimized.cbm


## 9. ROC-AUC Curves

In [20]:
# ROC-AUC curve analysis and visualization
print("📈 ROC-AUC Analysis and Visualization")
print("=" * 40)

# Generate ROC curves with detailed analysis
plot_AUC(
    df_train[target],  # Train true labels
    predictions["train"],  # Train predictions
    df_val[target],  # Validation true labels
    predictions["val"],  # Validation predictions
    df_test[target],  # Test true labels
    predictions["test"],  # Test predictions
)

📈 ROC-AUC Analysis and Visualization


## 10. Precision-Recall Curves

In [21]:
# Precision-Recall curve analysis and visualization
print("📊 Precision-Recall Analysis and Visualization")
print("=" * 45)

# Note: Using predictions dictionary instead of DataFrame columns
plot_PR(
    df_train[target],  # Train true labels
    predictions["train"],  # Train predictions
    df_val[target],  # Validation true labels
    predictions["val"],  # Validation predictions
    df_test[target],  # Test true labels
    predictions["test"],  # Test predictions
)

📊 Precision-Recall Analysis and Visualization


## Summary

✅ **Optimized Model**: Best hyperparameters from tuning  
✅ **Model Saved**: Ready for deployment  
⏭️ **Next**: Model interpretation with SHAP